In [3]:
# %%
"""
处理“扁平结构”的 JPG 图像 + CVAT XML（polyline 若为 2 点 -> 只画第 1 支蓝箭；若为 4 点 -> 先蓝后红），
并生成 3×3 角落大图（左上角为已绘箭头小图，其余全黑）。

文件名形如： 009004__images__step00000.jpg
XML: <polyline label="arrow" points="x1,y1; x2,y2; [x3,y3; x4,y4]">

—— 颜色（固定 RGB）：
蓝色 rgb(74, 161, 241) -> BGR(241,161,74)
红色 rgb(254, 0, 0)   -> BGR(0,0,254)

—— 箭头样式：
固定头大小（独立于线段长度），两臂相对主体方向 ±60°，类似于 cv2.arrowedLine 的“分叉箭头”，
但不使用比例尺寸，确保不同长度箭头的头部一致。
"""

from pathlib import Path
import re
import xml.etree.ElementTree as ET

import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# ========= 参数 =========
root_dir    = Path(r"E:\rt1threeslice\slice10_flat")                        # 扁平图像目录（含 00xxxx__images__step00000.jpg）
xml_path    = Path(r"E:\rt1threeslice\annotations_slice10.xml")  # 你的 CVAT XML
out_raw_dir = root_dir / "arrows_raw_ff"          # 仅绘箭后的小图目录
out_ff_dir  = root_dir / "ff_images"              # 3×3 角落大图目录（<六位id>_ff.jpg）
out_raw_dir.mkdir(parents=True, exist_ok=True)
out_ff_dir.mkdir(parents=True, exist_ok=True)

# ========= 颜色（RGB -> BGR for OpenCV）=========
RGB_BLUE = (74, 161, 241)
RGB_RED  = (254, 0, 0)
BLUE = (RGB_BLUE[2], RGB_BLUE[1], RGB_BLUE[0])  # (241,161,74)
RED  = (RGB_RED[2],  RGB_RED[1],  RGB_RED[0])   # (0,0,254)

# ========= 绘制参数 =========
LINE_THICKNESS = 2
HEAD_LENGTH_PX = 8   # 箭头臂的长度（像素，固定，与线长无关）
HEAD_ANGLE_DEG = 30   # 两臂相对箭身方向的偏转角（±60°）

# 文件名匹配：六位 id
PAT_JPG = re.compile(r"^(\d{6})__images__step\d+\.jpg$", re.IGNORECASE)

# ========= 工具：固定头大小的“分叉箭头” =========
def draw_fixed_head_arrow(img, p_tail, p_tip, color,
                          thickness=LINE_THICKNESS,
                          head_len_px=HEAD_LENGTH_PX,
                          head_angle_deg=HEAD_ANGLE_DEG):
    """
    在 img 上画分叉箭头（固定头大小）：
      - 箭身：tail -> tip
      - 箭头两臂：从 tip 出发，沿 tip->tail 方向旋转 ±head_angle_deg，长度 head_len_px
    """
    x0, y0 = float(p_tail[0]), float(p_tail[1])
    x1, y1 = float(p_tip[0]),  float(p_tip[1])

    v = np.array([x0 - x1, y0 - y1], dtype=np.float32)  # 从 tip 指回 tail
    L = np.linalg.norm(v)
    if L < 1e-6:
        return  # 零长度，跳过

    u = v / L
    theta = np.deg2rad(head_angle_deg)
    cos_t, sin_t = np.cos(theta), np.sin(theta)

    # 旋转矩阵 R(±θ) * u
    def rot(vec, sign):
        return np.array([cos_t * vec[0] - sign*sin_t * vec[1],
                         sign*sin_t * vec[0] + cos_t * vec[1]], dtype=np.float32)

    tip = np.array([x1, y1], dtype=np.float32)
    dir_left  = rot(u, +1)   # +θ
    dir_right = rot(u, -1)   # -θ

    p_left  = tip + dir_left  * head_len_px
    p_right = tip + dir_right * head_len_px

    # 主体
    cv2.line(img, (int(round(x0)), int(round(y0))),
                  (int(round(x1)), int(round(y1))),
                  color=color, thickness=thickness, lineType=cv2.LINE_AA)
    # 两臂
    cv2.line(img, (int(round(x1)), int(round(y1))),
                  (int(round(p_left[0])), int(round(p_left[1]))),
                  color=color, thickness=thickness, lineType=cv2.LINE_AA)
    cv2.line(img, (int(round(x1)), int(round(y1))),
                  (int(round(p_right[0])), int(round(p_right[1]))),
                  color=color, thickness=thickness, lineType=cv2.LINE_AA)

# ========= 解析 XML：允许 2 点或 4 点 =========
def parse_cvat_arrow_points(xml_file: Path):
    """
    返回 dict[name] = [(x1,y1), (x2,y2), ...]
    仅取 label="arrow" 的第一条 polyline。
    - 若为 2 点：只画第 1 支箭（蓝色）
    - 若为 4 点：画两支箭（蓝、红）
    - 若 >4：仅取前 4 点
    - 若 <2：跳过
    """
    tree = ET.parse(str(xml_file))
    root = tree.getroot()
    name2pts = {}

    for img_tag in root.findall(".//image"):
        name = img_tag.attrib.get("name", "").strip()
        if not name:
            continue

        polys = img_tag.findall(".//polyline")
        use_pts = None
        for poly in polys:
            if poly.attrib.get("label", "") != "arrows":
                continue
            pts_str = poly.attrib.get("points", "").strip()
            if not pts_str:
                continue
            parts = [p.strip() for p in pts_str.split(";") if p.strip()]
            pts = []
            ok = True
            for p in parts:
                try:
                    xs, ys = p.split(",")
                    pts.append((float(xs), float(ys)))
                except:
                    ok = False
                    break
            if ok and len(pts) >= 2:
                # 2 点或 4 点有效；>4 仅取前 4
                use_pts = pts[:4]
                break

        if use_pts is not None:
            name2pts[name] = use_pts

    return name2pts

# ========= 主流程 =========
name2pts = parse_cvat_arrow_points(xml_path)
print(f"[INFO] 解析到 {len(name2pts)} 条有效 polyline（≥2 点）。")

processed, skipped = 0, 0
for name in tqdm(sorted(name2pts.keys()), desc="Drawing & Building 3x3 FF"):
    img_path = root_dir / name
    if not img_path.exists():
        tqdm.write(f"[WARN] 磁盘缺图：{name}，跳过。")
        skipped += 1
        continue

    m = PAT_JPG.match(name)
    if not m:
        tqdm.write(f"[WARN] 文件名不符规则（需六位 id 前缀）：{name}，跳过。")
        skipped += 1
        continue
    six_id = m.group(1)

    pts = name2pts[name]
    if len(pts) < 2:
        tqdm.write(f"[WARN] 少于 2 点：{name}，跳过。")
        skipped += 1
        continue

    # 读图
    img = cv2.imread(str(img_path), cv2.IMREAD_COLOR)
    if img is None:
        tqdm.write(f"[WARN] 读取失败：{name}")
        skipped += 1
        continue

    # --- 画箭 ---
    # 第一支（蓝）：A->B
    A, B = pts[0], pts[1]
    draw_fixed_head_arrow(img, A, B, color=BLUE)

    # 第二支（红，若存在）：C->D
    if len(pts) >= 4:
        C, D = pts[2], pts[3]
        draw_fixed_head_arrow(img, C, D, color=RED)

    # 小图输出（仅绘箭）
    raw_out = out_raw_dir / f"{six_id}__arrow.jpg"
    if not cv2.imwrite(str(raw_out), img):
        tqdm.write(f"[WARN] 保存失败：{raw_out.name}")
        skipped += 1
        continue

    # 3×3 角落大图（左上角贴入小图，其余黑）
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    h, w = img_rgb.shape[:2]
    big = Image.new("RGB", (w*3, h*3), color=(0,0,0))
    big.paste(Image.fromarray(img_rgb), (0, 0))
    ff_out = out_ff_dir / f"{six_id}_ff.jpg"
    big.save(ff_out, quality=95)

    processed += 1

print(f"\n✅ 完成：成功 {processed}，跳过 {skipped}。")
print(f"   小图目录：{out_raw_dir}")
print(f"   3×3 目录：{out_ff_dir}")


[INFO] 解析到 983 条有效 polyline（≥2 点）。


Drawing & Building 3x3 FF: 100%|██████████| 983/983 [00:12<00:00, 77.61it/s]


✅ 完成：成功 983，跳过 0。
   小图目录：E:\rt1threeslice\slice10_flat\arrows_raw_ff
   3×3 目录：E:\rt1threeslice\slice10_flat\ff_images
